**AI System to Summarize YouTube Videos into Notes** 📹



Step 0: The Setup
First, let’s get our tools ready. We’ll use the Hugging Face transformers ecosystem, which is a standard for NLP. Here are the libraries you’ll need:

youtube-transcript-api: To scrape the subtitles.
transformers & accelerate: To run the AI model.
sentencepiece: A tokenizer required by the T5 model family.

In [1]:
!pip install -U youtube-transcript-api transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.2/485.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


Step 1: Extracting Transcripts

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
import re

def extract_video_id(url):
    """Extracts video ID from different YouTube URL formats."""
    # We use Regex to hunt for the 11-character ID after 'v=' or 'youtu.be/'
    match = re.search(r"(?:v=|youtu\.be/)([a-zA-Z0-9_-]{11})", url)
    return match.group(1) if match else None

def get_transcript(video_id):
    """Fetch transcript using the NEW API format."""
    try:
        api = YouTubeTranscriptApi()
        # The .fetch method grabs the subtitle object list
        transcript = api.fetch(video_id)
        # We join the list into a single long string of text
        return " ".join([t.text for t in transcript])

    except TranscriptsDisabled:
        return "Error: Transcripts are disabled for this video."
    except NoTranscriptFound:
        return "Error: No transcript found for this video."
    except Exception as e:
        return f"Error: {str(e)}"

Step 2: Loading the Model

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check if we have a GPU (CUDA) available to speed things up
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/flan-t5-base"

# Load the tokenizer (translates text to numbers)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model (the neural network) and move it to the GPU/CPU
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step 3: Summarization Function


In [4]:
def summarize_chunk(text_chunk):
    # We give the model a specific instruction (prompt engineering)
    prompt = f"Summarize the following text clearly:\n{text_chunk}"

    # Convert text to tensor numbers (inputs)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    # Generate the summary
    summary_ids = model.generate(
        **inputs,
        max_new_tokens=120,    # Max length of the summary
        num_beams=4,           # Look for the 4 best paths (higher quality)
        length_penalty=1.0,    # Balance between short and long
        early_stopping=True
    )

    # Decode back to text
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

Step 4: Chunking the Text

In [5]:
def chunk_text(text, chunk_size=1200):
    sentences = text.split(". ")
    chunks, current_chunk = [], ""

    for sentence in sentences:
        # Check if adding the next sentence exceeds our limit
        if len(current_chunk) + len(sentence) < chunk_size:
            current_chunk += sentence + ". "
        else:
            # If full, seal the chunk and start a new one
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

Step 5: Main Pipeline

In [8]:
def generate_video_notes(video_url):
    print(f"\n🎬 Processing video: {video_url}")

    video_id = extract_video_id(video_url)
    if not video_id:
        print("Invalid YouTube URL.")
        return

    print("🎧 Fetching transcript...")
    transcript = get_transcript(video_id)

    if transcript.startswith("Error"):
        print(transcript)
        return

    print("🔪 Chunking transcript...")
    chunks = chunk_text(transcript)
    print(f"   -> {len(chunks)} chunks created.")

    print("🧠 Generating AI notes...")
    notes = []

    # Loop through chunks and summarize each one
    for i, chunk in enumerate(chunks):
        print(f"   Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarize_chunk(chunk)
        notes.append(f"- {summary}")

    print("\n" + "="*50)
    print("📝 AI GENERATED NOTES")
    print("="*50)
    print("\n".join(notes))


if __name__ == "__main__":
    url = input("Paste YouTube URL: ")
    generate_video_notes(url)

Paste YouTube URL: https://youtu.be/0BY-6tJmz1g?si=ULVyX0WV-9UH4lwT

🎬 Processing video: https://youtu.be/0BY-6tJmz1g?si=ULVyX0WV-9UH4lwT
🎧 Fetching transcript...
🔪 Chunking transcript...
   -> 15 chunks created.
🧠 Generating AI notes...
   Summarizing chunk 1/15...
   Summarizing chunk 2/15...
   Summarizing chunk 3/15...
   Summarizing chunk 4/15...
   Summarizing chunk 5/15...
   Summarizing chunk 6/15...
   Summarizing chunk 7/15...
   Summarizing chunk 8/15...
   Summarizing chunk 9/15...
   Summarizing chunk 10/15...
   Summarizing chunk 11/15...
   Summarizing chunk 12/15...
   Summarizing chunk 13/15...
   Summarizing chunk 14/15...
   Summarizing chunk 15/15...

📝 AI GENERATED NOTES
- Understand how I am analyzing the hospital data set using Python.
- I'm going to go into Google Collab. I'm going to create a new notebook.
- Go to https://www.google.com/input.html in Google Gemini. Go to https://www.google.com/input.html in Google Gemini. Go to https://www.google.com/input.html